In [1]:
from collections import OrderedDict
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import sampler
import os
import torchvision.datasets as dset
import torchvision.transforms as T
import pandas as pd
# for plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

from trainer_functions.imagenetc import ImageNetC
from trainer_functions.imagenetctrainer import build_resnet50, evaluate_imagenetc, train_imagenetc

In [2]:
# corruption_list = ["brightness", "contrast", "defocus_blur", "elastic_transform", "fog", "frost", "gaussian_noise", "glass_blur",
# "impulse_noise", "jpeg_compression", "motion_blur", "pixelate", "shot_noise", "snow", "zoom_blur"]

""" corruption_list = ["frost", "gaussian_noise", "glass_blur", "impulse_noise", "snow"] """

corruption_list = ["brightness", "contrast", "defocus_blur", "elastic_transform", "fog", "jpeg_compression", "motion_blur", "pixelate", "shot_noise",  "zoom_blur"]

n_subset = 2000
# lrs = [1e-3, 1e-4, 1e-5]
lrs = [1e-3]

## writing fine tuning code here

In [3]:
# empty dataframe
df = pd.DataFrame(columns=['corruption', 'accuracy', "lr", "state", "n_subset"])

for corruption in corruption_list:
    dataset = ImageNetC(root='/Users/rada/Desktop/KAIST/Deep Learning/Datasets/ImageNetC',
                        corruption=corruption, severity=5,
                        transform=T.Compose([T.ToTensor()])
                        )

    indices = torch.randperm(len(dataset))
    for lr in lrs:

        ictrain = torch.utils.data.Subset(dataset, indices[:n_subset])
        icvalid = torch.utils.data.Subset(dataset, indices[n_subset:n_subset+2000])
        ictest = torch.utils.data.Subset(dataset, indices[n_subset+2000:n_subset+4000])

        ictrainload = DataLoader(ictrain, batch_size=64, shuffle=True)
        icvalidload = DataLoader(icvalid, batch_size=64, shuffle=True)
        ictestload = DataLoader(ictest, batch_size=64, shuffle=True)

        
        device = torch.device("mps")
        tune_net = build_resnet50(device)

        # first layer is fine-tuned
        optimizer = optim.Adam(tune_net.layer1.parameters(), lr=lr, weight_decay=0.0001)

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

        acc = train_imagenetc(tune_net, ictrainload, icvalidload, optimizer, scheduler, device=device, epochs=10)

        # adding results to dataframe
        dict = {}
        dict['corruption'] = corruption
        dict['accuracy'] = acc
        dict['lr'] = lr
        dict['state'] = "first layer" # change when run
        dict['n_subset'] = n_subset
        df_temp = pd.DataFrame(dict, index=[0])
        df = pd.concat([df, df_temp])

# change when run
df.to_csv("imageNetC_layer1_corruptionsLeft.csv")

Building model...
Epoch 0, loss = 2.1975, lr 0.00100000
Got 1432 / 2000 correct (71.60)

Epoch 1, loss = 0.8567, lr 0.00100000
Got 1447 / 2000 correct (72.35)

Epoch 2, loss = 1.5762, lr 0.00100000
Got 1468 / 2000 correct (73.40)

Epoch 3, loss = 0.9671, lr 0.00100000
Got 1458 / 2000 correct (72.90)

Epoch 4, loss = 1.4019, lr 0.00100000
Got 1467 / 2000 correct (73.35)

Epoch 5, loss = 1.4616, lr 0.00100000
Got 1478 / 2000 correct (73.90)

Epoch 6, loss = 1.4652, lr 0.00100000
Got 1470 / 2000 correct (73.50)

Epoch 7, loss = 1.2811, lr 0.00100000
Got 1478 / 2000 correct (73.90)

Epoch 8, loss = 1.0664, lr 0.00100000
Got 1487 / 2000 correct (74.35)

Epoch 9, loss = 1.2782, lr 0.00100000
Got 1454 / 2000 correct (72.70)

Building model...
Epoch 0, loss = 4.7683, lr 0.00100000
Got 145 / 2000 correct (7.25)

Epoch 1, loss = 3.7244, lr 0.00100000
Got 649 / 2000 correct (32.45)

Epoch 2, loss = 2.1269, lr 0.00100000
Got 820 / 2000 correct (41.00)

Epoch 3, loss = 2.7596, lr 0.00100000
Got 799

In [4]:
# empty dataframe
df = pd.DataFrame(columns=['corruption', 'accuracy', "lr", "state", "n_subset"])

for corruption in corruption_list:
    dataset = ImageNetC(root='/Users/rada/Desktop/KAIST/Deep Learning/Datasets/ImageNetC',
                        corruption=corruption, severity=5,
                        transform=T.Compose([T.ToTensor()])
                        )

    indices = torch.randperm(len(dataset))
    for lr in lrs:

        ictrain = torch.utils.data.Subset(dataset, indices[:n_subset])
        icvalid = torch.utils.data.Subset(dataset, indices[n_subset:n_subset+2000])
        ictest = torch.utils.data.Subset(dataset, indices[n_subset+2000:n_subset+4000])

        ictrainload = DataLoader(ictrain, batch_size=64, shuffle=True)
        icvalidload = DataLoader(icvalid, batch_size=64, shuffle=True)
        ictestload = DataLoader(ictest, batch_size=64, shuffle=True)

        
        device = torch.device("mps")
        tune_net = build_resnet50(device)

        # second layer is fine-tuned
        optimizer = optim.Adam(tune_net.layer2.parameters(), lr=lr, weight_decay=0.0001)

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

        acc = train_imagenetc(tune_net, ictrainload, icvalidload, optimizer, scheduler, device=device, epochs=10)

        # adding results to dataframe
        dict = {}
        dict['corruption'] = corruption
        dict['accuracy'] = acc
        dict['lr'] = lr
        dict['state'] = "second layer" # change when run
        dict['n_subset'] = n_subset
        df_temp = pd.DataFrame(dict, index=[0])
        df = pd.concat([df, df_temp])

# change when run
df.to_csv("imageNetC_layer2_corruptionsLeft.csv")

Building model...
Epoch 0, loss = 1.7453, lr 0.00100000
Got 1440 / 2000 correct (72.00)

Epoch 1, loss = 0.9831, lr 0.00100000
Got 1455 / 2000 correct (72.75)

Epoch 2, loss = 0.9619, lr 0.00100000
Got 1463 / 2000 correct (73.15)

Epoch 3, loss = 0.6695, lr 0.00100000
Got 1462 / 2000 correct (73.10)

Epoch 4, loss = 0.8172, lr 0.00100000
Got 1459 / 2000 correct (72.95)

Epoch 5, loss = 0.6348, lr 0.00100000
Got 1442 / 2000 correct (72.10)

Epoch 6, loss = 0.3897, lr 0.00100000
Got 1449 / 2000 correct (72.45)

Epoch 7, loss = 0.3467, lr 0.00100000
Got 1423 / 2000 correct (71.15)

Epoch 8, loss = 0.3954, lr 0.00100000
Got 1448 / 2000 correct (72.40)

Epoch 9, loss = 0.2378, lr 0.00100000
Got 1435 / 2000 correct (71.75)

Building model...
Epoch 0, loss = 4.0932, lr 0.00100000
Got 181 / 2000 correct (9.05)

Epoch 1, loss = 3.2131, lr 0.00100000
Got 514 / 2000 correct (25.70)

Epoch 2, loss = 2.5189, lr 0.00100000
Got 842 / 2000 correct (42.10)

Epoch 3, loss = 2.6977, lr 0.00100000
Got 832

In [5]:
# empty dataframe
df = pd.DataFrame(columns=['corruption', 'accuracy', "lr", "state", "n_subset"])

for corruption in corruption_list:
    dataset = ImageNetC(root='/Users/rada/Desktop/KAIST/Deep Learning/Datasets/ImageNetC',
                        corruption=corruption, severity=5,
                        transform=T.Compose([T.ToTensor()])
                        )

    indices = torch.randperm(len(dataset))
    for lr in lrs:

        ictrain = torch.utils.data.Subset(dataset, indices[:n_subset])
        icvalid = torch.utils.data.Subset(dataset, indices[n_subset:n_subset+2000])
        ictest = torch.utils.data.Subset(dataset, indices[n_subset+2000:n_subset+4000])

        ictrainload = DataLoader(ictrain, batch_size=64, shuffle=True)
        icvalidload = DataLoader(icvalid, batch_size=64, shuffle=True)
        ictestload = DataLoader(ictest, batch_size=64, shuffle=True)

        
        device = torch.device("mps")
        tune_net = build_resnet50(device)

        # third layer is fine-tuned
        optimizer = optim.Adam(tune_net.layer3.parameters(), lr=lr, weight_decay=0.0001)

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

        acc = train_imagenetc(tune_net, ictrainload, icvalidload, optimizer, scheduler, device=device, epochs=10)

        # adding results to dataframe
        dict = {}
        dict['corruption'] = corruption
        dict['accuracy'] = acc
        dict['lr'] = lr
        dict['state'] = "third layer" # change when run
        dict['n_subset'] = n_subset
        df_temp = pd.DataFrame(dict, index=[0])
        df = pd.concat([df, df_temp])

# change when run
df.to_csv("imageNetC_layer3_corruptionsLeft.csv")

Building model...
Epoch 0, loss = 2.4056, lr 0.00100000
Got 1234 / 2000 correct (61.70)

Epoch 1, loss = 0.5243, lr 0.00100000
Got 1280 / 2000 correct (64.00)

Epoch 2, loss = 0.7194, lr 0.00100000
Got 1264 / 2000 correct (63.20)

Epoch 3, loss = 0.2259, lr 0.00100000
Got 1269 / 2000 correct (63.45)

Epoch 4, loss = 0.1389, lr 0.00100000
Got 1270 / 2000 correct (63.50)

Epoch 5, loss = 0.1270, lr 0.00100000
Got 1280 / 2000 correct (64.00)

Epoch 6, loss = 0.0786, lr 0.00100000
Got 1276 / 2000 correct (63.80)

Epoch 7, loss = 0.0476, lr 0.00100000
Got 1285 / 2000 correct (64.25)

Epoch 8, loss = 0.0673, lr 0.00100000
Got 1289 / 2000 correct (64.45)

Epoch 9, loss = 0.0602, lr 0.00100000
Got 1289 / 2000 correct (64.45)

Building model...
Epoch 0, loss = 3.2980, lr 0.00100000
Got 217 / 2000 correct (10.85)

Epoch 1, loss = 2.6106, lr 0.00100000
Got 519 / 2000 correct (25.95)

Epoch 2, loss = 1.4515, lr 0.00100000
Got 639 / 2000 correct (31.95)

Epoch 3, loss = 0.8577, lr 0.00100000
Got 64

In [6]:
# empty dataframe
df = pd.DataFrame(columns=['corruption', 'accuracy', "lr", "state", "n_subset"])

for corruption in corruption_list:
    dataset = ImageNetC(root='/Users/rada/Desktop/KAIST/Deep Learning/Datasets/ImageNetC',
                        corruption=corruption, severity=5,
                        transform=T.Compose([T.ToTensor()])
                        )

    indices = torch.randperm(len(dataset))
    for lr in lrs:

        ictrain = torch.utils.data.Subset(dataset, indices[:n_subset])
        icvalid = torch.utils.data.Subset(dataset, indices[n_subset:n_subset+2000])
        ictest = torch.utils.data.Subset(dataset, indices[n_subset+2000:n_subset+4000])

        ictrainload = DataLoader(ictrain, batch_size=64, shuffle=True)
        icvalidload = DataLoader(icvalid, batch_size=64, shuffle=True)
        ictestload = DataLoader(ictest, batch_size=64, shuffle=True)

        
        device = torch.device("mps")
        tune_net = build_resnet50(device)

        # fourth layer is fine-tuned
        optimizer = optim.Adam(tune_net.layer4.parameters(), lr=lr, weight_decay=0.0001)

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

        acc = train_imagenetc(tune_net, ictrainload, icvalidload, optimizer, scheduler, device=device, epochs=10)

        # adding results to dataframe
        dict = {}
        dict['corruption'] = corruption
        dict['accuracy'] = acc
        dict['lr'] = lr
        dict['state'] = "fourth layer" # change when run
        dict['n_subset'] = n_subset
        df_temp = pd.DataFrame(dict, index=[0])
        df = pd.concat([df, df_temp])

# change when run
df.to_csv("imageNetC_layer4_corruptionsLeft.csv")

Building model...
Epoch 0, loss = 3.2399, lr 0.00100000
Got 1058 / 2000 correct (52.90)

Epoch 1, loss = 0.9323, lr 0.00100000
Got 1062 / 2000 correct (53.10)

Epoch 2, loss = 0.2273, lr 0.00100000
Got 1054 / 2000 correct (52.70)

Epoch 3, loss = 0.0290, lr 0.00100000
Got 1072 / 2000 correct (53.60)

Epoch 4, loss = 0.0448, lr 0.00100000
Got 1051 / 2000 correct (52.55)

Epoch 5, loss = 0.0079, lr 0.00100000
Got 1054 / 2000 correct (52.70)

Epoch 6, loss = 0.0149, lr 0.00100000
Got 1061 / 2000 correct (53.05)

Epoch 7, loss = 0.0297, lr 0.00100000
Got 1065 / 2000 correct (53.25)

Epoch 8, loss = 0.0188, lr 0.00100000
Got 1068 / 2000 correct (53.40)

Epoch 9, loss = 0.1190, lr 0.00100000
Got 1071 / 2000 correct (53.55)

Building model...
Epoch 0, loss = 5.4986, lr 0.00100000
Got 144 / 2000 correct (7.20)

Epoch 1, loss = 3.1807, lr 0.00100000
Got 304 / 2000 correct (15.20)

Epoch 2, loss = 1.9253, lr 0.00100000
Got 316 / 2000 correct (15.80)

Epoch 3, loss = 0.6965, lr 0.00100000
Got 299

In [7]:
# empty dataframe
df = pd.DataFrame(columns=['corruption', 'accuracy', "lr", "state", "n_subset"])

for corruption in corruption_list:
    dataset = ImageNetC(root='/Users/rada/Desktop/KAIST/Deep Learning/Datasets/ImageNetC',
                        corruption=corruption, severity=5,
                        transform=T.Compose([T.ToTensor()])
                        )

    indices = torch.randperm(len(dataset))
    for lr in lrs:

        ictrain = torch.utils.data.Subset(dataset, indices[:n_subset])
        icvalid = torch.utils.data.Subset(dataset, indices[n_subset:n_subset+2000])
        ictest = torch.utils.data.Subset(dataset, indices[n_subset+2000:n_subset+4000])

        ictrainload = DataLoader(ictrain, batch_size=64, shuffle=True)
        icvalidload = DataLoader(icvalid, batch_size=64, shuffle=True)
        ictestload = DataLoader(ictest, batch_size=64, shuffle=True)

        
        device = torch.device("mps")
        tune_net = build_resnet50(device)

        # last layer is fine-tuned
        optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr, weight_decay=0.0001)

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

        acc = train_imagenetc(tune_net, ictrainload, icvalidload, optimizer, scheduler, device=device, epochs=10)

        # adding results to dataframe
        dict = {}
        dict['corruption'] = corruption
        dict['accuracy'] = acc
        dict['lr'] = lr
        dict['state'] = "fully-connected layer" # change when run
        dict['n_subset'] = n_subset
        df_temp = pd.DataFrame(dict, index=[0])
        df = pd.concat([df, df_temp])

# change when run
df.to_csv("imageNetC_fclayer_corruptionsLeft.csv")

Building model...
Epoch 0, loss = 1.5322, lr 0.00100000
Got 1320 / 2000 correct (66.00)

Epoch 1, loss = 0.6928, lr 0.00100000
Got 1356 / 2000 correct (67.80)

Epoch 2, loss = 0.5394, lr 0.00100000
Got 1323 / 2000 correct (66.15)

Epoch 3, loss = 0.3087, lr 0.00100000
Got 1323 / 2000 correct (66.15)

Epoch 4, loss = 0.4336, lr 0.00100000
Got 1317 / 2000 correct (65.85)

Epoch 5, loss = 0.2624, lr 0.00100000
Got 1302 / 2000 correct (65.10)

Epoch 6, loss = 0.3964, lr 0.00100000
Got 1303 / 2000 correct (65.15)

Epoch 7, loss = 0.2399, lr 0.00100000
Got 1280 / 2000 correct (64.00)

Epoch 8, loss = 0.1192, lr 0.00100000
Got 1275 / 2000 correct (63.75)

Epoch 9, loss = 0.2700, lr 0.00100000
Got 1274 / 2000 correct (63.70)

Building model...
Epoch 0, loss = 6.3076, lr 0.00100000
Got 163 / 2000 correct (8.15)

Epoch 1, loss = 5.1504, lr 0.00100000
Got 296 / 2000 correct (14.80)

Epoch 2, loss = 3.2683, lr 0.00100000
Got 317 / 2000 correct (15.85)

Epoch 3, loss = 2.8790, lr 0.00100000
Got 282

In [8]:
# empty dataframe
df = pd.DataFrame(columns=['corruption', 'accuracy', "lr", "state", "n_subset"])

for corruption in corruption_list:
    dataset = ImageNetC(root='/Users/rada/Desktop/KAIST/Deep Learning/Datasets/ImageNetC',
                        corruption=corruption, severity=5,
                        transform=T.Compose([T.ToTensor()])
                        )

    indices = torch.randperm(len(dataset))
    for lr in lrs:

        data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

        ictrain = torch.utils.data.Subset(dataset, indices[:n_subset])
        icvalid = torch.utils.data.Subset(dataset, indices[n_subset:n_subset+2000])
        ictest = torch.utils.data.Subset(dataset, indices[n_subset+2000:n_subset+4000])

        ictrainload = DataLoader(ictrain, batch_size=64, shuffle=True)
        icvalidload = DataLoader(icvalid, batch_size=64, shuffle=True)
        ictestload = DataLoader(ictest, batch_size=64, shuffle=True)

        
        device = torch.device("mps")
        tune_net = build_resnet50(device)

        # all layers are tuned
        optimizer = optim.Adam(tune_net.parameters(), lr=lr, weight_decay=0.0001)

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

        acc = train_imagenetc(tune_net, ictrainload, icvalidload, optimizer, scheduler, device=device, epochs=10)

        # adding results to dataframe
        dict = {}
        dict['corruption'] = corruption
        dict['accuracy'] = acc
        dict['lr'] = lr
        dict['state'] = "all layers" # change when run
        dict['n_subset'] = n_subset
        df_temp = pd.DataFrame(dict, index=[0])
        df = pd.concat([df, df_temp])

# change when run
df.to_csv("imageNetC_allLayers_corruptionsLeft.csv")

Building model...
Epoch 0, loss = 3.4851, lr 0.00100000
Got 519 / 2000 correct (25.95)

Epoch 1, loss = 0.9323, lr 0.00100000
Got 562 / 2000 correct (28.10)

Epoch 2, loss = 0.2053, lr 0.00100000
Got 638 / 2000 correct (31.90)

Epoch 3, loss = 0.4976, lr 0.00100000
Got 686 / 2000 correct (34.30)

Epoch 4, loss = 0.0844, lr 0.00100000
Got 631 / 2000 correct (31.55)

Epoch 5, loss = 0.0988, lr 0.00100000
Got 598 / 2000 correct (29.90)

Epoch 6, loss = 0.2632, lr 0.00100000
Got 522 / 2000 correct (26.10)

Epoch 7, loss = 0.3264, lr 0.00100000
Got 500 / 2000 correct (25.00)

Epoch 8, loss = 0.7513, lr 0.00100000
Got 344 / 2000 correct (17.20)

Epoch 9, loss = 0.6092, lr 0.00100000
Got 345 / 2000 correct (17.25)

Building model...
Epoch 0, loss = 3.9909, lr 0.00100000
Got 137 / 2000 correct (6.85)

Epoch 1, loss = 1.1419, lr 0.00100000
Got 97 / 2000 correct (4.85)

Epoch 2, loss = 0.8563, lr 0.00100000
Got 245 / 2000 correct (12.25)

Epoch 3, loss = 0.3964, lr 0.00100000
Got 173 / 2000 corr